# MINES ParisTech - PSL - Semestre 2 - Traitement du Signal
# Mini-projet groupe 2 : interpolation de données manquantes
# Judith Bellon & Louis-Justin Tallot
Contexte : dans le cadre d'acquisition de données sismiques, des capteurs (récepteurs) sont positionnés le long d'une ligne à la surface de la terre. Chacun enregistre au cours du temps le deplacement des particules (signal oscillant) qui varie lors du passage d'une onde acoustique ou élastique. Cet enregistrement s'appelle une **trace**.

En pratique, il arrive que certains capteurs sont défaillants ou encore qu'ils ne sont pas présents dans certaines zones (pour des problèmes d'accessibilité par exemple). L'objectif principal du mini-projet est de reconstituer les traces manquantes.

Hypotheses : nous ferons l'hypothèse importante que les **signaux correspondent à la superposition lineaire d'ondes planes** dans les enregistrements (voir l'exemple fourni plus bas avec 3 événements).

Notions abordées : 
* Analyse des signaux après transformée de Fourier 2d
* Analyse de signaux aliasés
* Régularisation et formulation sous forme de problème inverse

Pour cela, deux approches sont considerées :
* Approche 1 : transformée de Fourier 2d, identification des événements principaux, définition d'un masque et reconstitution des traces manquantes
* Approche 2 : suite de l'approche 1 pour s'assurer que la prédiction respecte bien les observables. Elle se fait au travers de la résolution d'un problème inverse avec ajout de régularisation

Attentes :
* Développement des approches 1 et 2 sur des données fournies ici
* Discusssion des limites des approches sur des exemples que vous pourrez créer vous-mêmes

Consignes :
* **Très important** : pour toutes les figures, bien indiquer les axes avec les bonnes unités et des labels de taille lisibles

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Pour la question 4
from scipy.optimize import minimize

In [ ]:
# Labelsize (display)
labelsize   = 14
    
# Load the input data
# Input data (with missing traces)
gpanel = np.load('gpanel.npy')
# Dense data (with all traces, only for comparison)
panel_dense = np.load('panel_dense.npy')
nt, nx = gpanel.shape

# Vertical axis -- time
dt   = 3.125e-3  # increment (s)
at   = np.linspace(0, dt*(nt-1), nt)
# Horizontal axis -- space
dx   = 10. # increment (m)
ax   = np.linspace(0, dx*(nx-1), nx)

In [ ]:
def display_data(data, titre="", x_label="Position x (m)", y_label="Time (s)", extent=None):
    fig = plt.figure()
    av = plt.subplot(111)
    
    if extent is not None:
        plt.imshow(data, extent=extent, aspect="auto")
    else:
        plt.imshow(data, aspect="auto")
        
    plt.title(titre, fontsize=14)
    av.set_ylabel(y_label, fontsize=14)
    av.set_xlabel(x_label, fontsize=14)
    vmax = np.max(np.abs(data))
    plt.clim(-vmax,vmax)
    plt.set_cmap('bwr')    

In [ ]:
display_data(gpanel, 
             titre='Observables (avec traces manquantes mises à zéro)',
            extent=[ax[0],ax[-1],at[-1],at[0]]
)

display_data(panel_dense, 
             titre='Observables (seulement pour comparaison)',
            extent=[ax[0],ax[-1],at[-1],at[0]]
)

# Question 1 : analyse des données dans le domaine de Fourier 2d
Représenter la transformée de Fourier 2d et bien préciser les axes. Indiquer à quoi correspond la fréquence maximale et le nombre d'onde maximal. Faire le lien entre les événements dans l'espace $(t,x)$ et ceux dans l'espace de Fourier. Expliquer ce qu'il se passe aux bords dans le domaine de Fourier. Ne pas hésiter à zoomer dans le domaine de Fourier.

In [ ]:
tf = np.fft.fftshift(np.fft.fft2(gpanel))
af1 = np.fft.fftshift(np.fft.fftfreq(nx, d=dx))
af2 = np.fft.fftshift(np.fft.fftfreq(nt, d=dt))

In [ ]:
display_data(np.abs(tf), 
             titre='Transformée de Fourier (avec traces manquantes mises à zéro)',
             x_label="Nombre d'onde (m$^{-1}$)",
             y_label="Fréquence (s$^{-1}$)",
             extent=[af1[0],af1[-1],af2[0],af2[-1]]
)
display_data(np.abs(tf), 
             titre='Transformée de Fourier (avec traces manquantes mises à zéro)\nZoom sur la partie centrale significative',
             x_label="Nombre d'onde (m$^{-1}$)",
             y_label="Fréquence (s$^{-1}$)",
             extent=[af1[0],af1[-1],af2[0],af2[-1]]
)
plt.ylim(-20, 20);

# Question 2 : définition d'un masque dans le domaine de Fourier 2d
On note $f$ la fréquence et $k_x$ le nombre d'onde. Dans le domaine de Fourier 2d, on souhaite definir un masque avec des 1 qui couvrent les événements principaux. On va supposer que les événements dans le domaine $(t,x)$ sont linéaires. Montrer qu'il en est de même dans le domaine $(f,k_x)$. A partir de $(f,k_x)=(0,0)$, faire des sommations selon différentes pentes (dans le domaine de Fourier, apres avoir pris le module des valeurs complexes) et tracer la valeur de ces sommes en fonction de la pente. On s'attend à ce que 3 valeurs ressortent. Calculer une fonction qui sélectionne automatiquement ces valeurs et construire un masque dans le domaine de Fourier 2d qui vaut 1 autour de ces trois valeurs principales et 0 en dehors.